**Gathering** data is the first step in data wrangling. Before gathering, we have no data, and after it, we do.

Gathering data varies from project to project. Sometimes you're just given data, or pointed to it like I've done for you throughout this course. Sometimes you need to search for the right data for your project. Sometimes the data you need isn't readily available, and you need to generate it yourself somehow. When you do find your data, it's not unusual for it to be spread across several different sources and file formats, which makes things tricky when organizing the data in your programming environment.

(:ᘌꇤ⁐ꃳ 三

## Flat files
Flat files contain tabular data in plain text format with one data record per line and each record or line having one or more fields. These fields are separated by delimiters, like commas, tabs, or colons.

#### Advantages of flat files include:

- They're text files and therefore human readable.
- Lightweight.
- Simple to understand.
- Software that can read/write text files is ubiquitous, like text editors.
- Great for small datasets.
#### Disadvantages of flat files, in comparison to relational databases, for example, include:

- Lack of standards.
- Data redundancy.
- Sharing data can be cumbersome.
- Not great for large datasets (see "When does small become large?" in the Cornell link in More Information).
#### some example
1. tsv,csv files are flat files 
2. txt file could be flat file ,and could not
3. excel file is not flat, excel has data in a zipped collection of XML files

In [136]:
import pandas as pd

In [137]:
df = pd.read_csv('bestofrt.tsv',sep = '\t')

In [138]:
df.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


In [139]:
df.to_csv('bestofrt_critical.csv')

### The two main ways to work with HTML files are:

- Saving the HTML file to your computer (using the Requests library for example) library and reading that file into a BeautifulSoup constructor
- Reading the HTML response content directly into a BeautifulSoup constructor (again using the Requests library for example)

For this lesson, you’re going to do neither of these. I've downloaded all of the Rotten Tomatoes HTML files for you and put them in a folder called rt_html in the Jupyter Notebooks in the Udacity classroom. If you want to work outside of the classroom, download this zip file and extract the rt_html folder. I recommend that you do and open the HTML files in your preferred text editor (e.g. Sublime, which is free) to inspect the HTML for the quizzes ahead.

## [Web scraper ethic](https://towardsdatascience.com/ethics-in-web-scraping-b96b18136f01?)

In [140]:
from bs4 import BeautifulSoup

In [141]:
with open('rt_html/et_the_extraterrestrial.html') as file:
    soup = BeautifulSoup(file, 'lxml')

In [142]:
soup

<!DOCTYPE html>
<html lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<script src="//cdn.optimizely.com/js/594670329.js"></script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="VPPXtECgUUeuATBacnqnCm4ydGO99reF-xgNklSbNbc" name="google-site-verification"/>
<meta content="034F16304017CA7DCF45D43850915323" name="msvalidate.01"/>
<link href="https://staticv2-4.rottentomatoes.com/static/images/iphone/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="https://staticv2-4.rottentomatoes.com/static/images/icons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://staticv2-4.rottentomatoes.com/static/styles/css/rt_main.css" rel="stylesheet"/>
<script id="jsonLdSchema" type="application/ld+json">{"@context":"http

It looks like a html file, acutally we can easily find and extract data from this HTML file with beautifulsoup functions

Fist find "title"

In [143]:
soup.find('title')

<title>E.T. The Extra-Terrestrial (1982) - Rotten Tomatoes</title>

We can use a .contents to return a list of the tags children

In [144]:
soup.find('title').contents[0]

'E.T. The Extra-Terrestrial\xa0(1982) - Rotten Tomatoes'

In [145]:
soup.find('title').contents[0][: - len(' - Rotten Tomatoes')]

'E.T. The Extra-Terrestrial\xa0(1982)'

In [146]:
soup.find('div',class_ = 'audience-score meter').find('span').contents[0][:-1]

'72'

In [147]:
a = soup.find('div',class_ = 'audience-info hidden-xs superPageFontColor').\
find_all('div')[1].contents[2].strip().replace(',','')
a



'32313030'

With your knowledge of HTML file structure, you're going to use Beautiful Soup to extract our desired Audience Score metric and number of audience ratings, along with the movie title like in the video above (so we have something to merge the datasets on later) for each HTML file, then save them in a pandas DataFrame.

The Jupyter Notebook below contains template code that:

- Creates an empty list, df_list, to which dictionaries will be appended. This list of dictionaries will eventually be converted to a pandas DataFrame (this is the [most efficient way of building a DataFrame row by row](https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop/28058264#28058264)).
- Loops through each movie's Rotten Tomatoes HTML file in the rt_html folder.
- Opens each HTML file and passes it into a file handle called file.
- Creates a DataFrame called df by converting df_list using the pd.DataFrame constructor.

In [148]:
os.listdir(folder)
os.path.join(folder, movie_html)

'rt_html/star_trek_11.html'

In [149]:
from bs4 import BeautifulSoup
import os
import pandas as pd

In [150]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'rt_html'
for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        soup = BeautifulSoup(file,'lxml')
        title = soup.find('title').contents[0][: - len(' - Rotten Tomatoes')]
        audience_score = soup.find('div',class_ = 'audience-score meter').find('span').contents[0][:-1]
        num_audience_ratings = soup.find('div',class_ = 'audience-info hidden-xs superPageFontColor').\
            find_all('div')[1].contents[2].replace(',','')
        
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})

df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])
df.head()

,title,audience_score,number_of_audience_ratings
0,Zootopia (2016),92,98633
1,The Treasure of the Sierra Madre (1948),93,25627
2,All Quiet on the Western Front (1930),89,17768
3,Rear Window (1954),95,149458
4,Selma (2015),86,60533


In [151]:
df.tail()

,title,audience_score,number_of_audience_ratings
95,The Night of the Hunter (1955),90,24322
96,A Streetcar Named Desire (1951),90,54761
97,Arrival (2016),82,78740
98,Skyfall (2012),86,372497
99,Star Trek (2009),91,746458


\xa0 Unicode representing spaces

you should be able to use unicodedata.normalize now

## Solution Test
Run the cell below the see if your solution is correct. If an `AssertionError` is thrown, your solution is incorrect. If no error is thrown, your solution is correct.

In [152]:
df_solution = pd.read_pickle('df_solution.pkl')
df.sort_values('title', inplace = True)
df.reset_index(inplace = True, drop = True)
df_solution.sort_values('title', inplace = True)
df_solution.reset_index(inplace = True, drop = True)
pd.testing.assert_frame_equal(df, df_solution)

In [153]:
df.to_csv('bestofrt_audience.csv')

## join the two dataset together

In [154]:
df1 = pd.read_csv('bestofrt_critical.csv')
df2 = pd.read_csv('bestofrt_audience.csv')

## Source: Downloading Files from the Internet

#### HTTP (Hypertext Transfer Protocol)
HTTP, the Hypertext Transfer Protocol, is the language that web browsers (like Chrome or Safari) and web servers (basically computers where the contents of a website are stored) speak to each other. Every time you open a web page, or download a file, or watch a video, it's HTTP that makes it possible.

#### HTTP is a request/response protocol:

- Your computer, a.k.a. the client, sends a request to a server for some file. For this lesson: "Get me the file 1-the-wizard-of-oz-1939-film.txt", for example. GET is the name of the HTTP request method (of which there are multiple) used for retrieving data.
- The web server sends back a response. If the request is valid: "Here is the file you asked for:", then followed by the contents of the 1-the-wizard-of-oz-1939-film.txt file itself.


In [155]:
import requests
import os

In [156]:
ebert_review_urls = ['https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9900_1-the-wizard-of-oz-1939-film/1-the-wizard-of-oz-1939-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9901_2-citizen-kane/2-citizen-kane.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9901_3-the-third-man/3-the-third-man.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9902_4-get-out-film/4-get-out-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9902_5-mad-max-fury-road/5-mad-max-fury-road.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9902_6-the-cabinet-of-dr.-caligari/6-the-cabinet-of-dr.-caligari.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9903_7-all-about-eve/7-all-about-eve.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9903_8-inside-out-2015-film/8-inside-out-2015-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9903_9-the-godfather/9-the-godfather.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_10-metropolis-1927-film/10-metropolis-1927-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_11-e.t.-the-extra-terrestrial/11-e.t.-the-extra-terrestrial.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_12-modern-times-film/12-modern-times-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_14-singin-in-the-rain/14-singin-in-the-rain.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9905_15-boyhood-film/15-boyhood-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9905_16-casablanca-film/16-casablanca-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9905_17-moonlight-2016-film/17-moonlight-2016-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9906_18-psycho-1960-film/18-psycho-1960-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9906_19-laura-1944-film/19-laura-1944-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9906_20-nosferatu/20-nosferatu.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9907_21-snow-white-and-the-seven-dwarfs-1937-film/21-snow-white-and-the-seven-dwarfs-1937-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9907_22-a-hard-day27s-night-film/22-a-hard-day27s-night-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9907_23-la-grande-illusion/23-la-grande-illusion.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9908_25-the-battle-of-algiers/25-the-battle-of-algiers.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9908_26-dunkirk-2017-film/26-dunkirk-2017-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9908_27-the-maltese-falcon-1941-film/27-the-maltese-falcon-1941-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9909_29-12-years-a-slave-film/29-12-years-a-slave-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9909_30-gravity-2013-film/30-gravity-2013-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9909_31-sunset-boulevard-film/31-sunset-boulevard-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990a_32-king-kong-1933-film/32-king-kong-1933-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990a_33-spotlight-film/33-spotlight-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990a_34-the-adventures-of-robin-hood/34-the-adventures-of-robin-hood.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990b_35-rashomon/35-rashomon.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990b_36-rear-window/36-rear-window.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990b_37-selma-film/37-selma-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990c_38-taxi-driver/38-taxi-driver.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990c_39-toy-story-3/39-toy-story-3.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990c_40-argo-2012-film/40-argo-2012-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_41-toy-story-2/41-toy-story-2.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_42-the-big-sick/42-the-big-sick.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_43-bride-of-frankenstein/43-bride-of-frankenstein.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_44-zootopia/44-zootopia.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990e_45-m-1931-film/45-m-1931-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990e_46-wonder-woman-2017-film/46-wonder-woman-2017-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990e_48-alien-film/48-alien-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990f_49-bicycle-thieves/49-bicycle-thieves.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990f_50-seven-samurai/50-seven-samurai.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990f_51-the-treasure-of-the-sierra-madre-film/51-the-treasure-of-the-sierra-madre-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9910_52-up-2009-film/52-up-2009-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9910_53-12-angry-men-1957-film/53-12-angry-men-1957-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9910_54-the-400-blows/54-the-400-blows.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9911_55-logan-film/55-logan-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9911_57-army-of-shadows/57-army-of-shadows.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9912_58-arrival-film/58-arrival-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9912_59-baby-driver/59-baby-driver.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_60-a-streetcar-named-desire-1951-film/60-a-streetcar-named-desire-1951-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_61-the-night-of-the-hunter-film/61-the-night-of-the-hunter-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_62-star-wars-the-force-awakens/62-star-wars-the-force-awakens.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_63-manchester-by-the-sea-film/63-manchester-by-the-sea-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9914_64-dr.-strangelove/64-dr.-strangelove.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9914_66-vertigo-film/66-vertigo-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9914_67-the-dark-knight-film/67-the-dark-knight-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9915_68-touch-of-evil/68-touch-of-evil.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9915_69-the-babadook/69-the-babadook.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9915_72-rosemary27s-baby-film/72-rosemary27s-baby-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9916_73-finding-nemo/73-finding-nemo.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9916_74-brooklyn-film/74-brooklyn-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9917_75-the-wrestler-2008-film/75-the-wrestler-2008-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9917_77-l.a.-confidential-film/77-l.a.-confidential-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9918_78-gone-with-the-wind-film/78-gone-with-the-wind-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9918_79-the-good-the-bad-and-the-ugly/79-the-good-the-bad-and-the-ugly.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9918_80-skyfall/80-skyfall.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_82-tokyo-story/82-tokyo-story.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_83-hell-or-high-water-film/83-hell-or-high-water-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_84-pinocchio-1940-film/84-pinocchio-1940-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_85-the-jungle-book-2016-film/85-the-jungle-book-2016-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991a_86-la-la-land-film/86-la-la-land-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991b_87-star-trek-film/87-star-trek-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991b_89-apocalypse-now/89-apocalypse-now.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991c_90-on-the-waterfront/90-on-the-waterfront.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991c_91-the-wages-of-fear/91-the-wages-of-fear.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991c_92-the-last-picture-show/92-the-last-picture-show.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991d_93-harry-potter-and-the-deathly-hallows-part-2/93-harry-potter-and-the-deathly-hallows-part-2.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991d_94-the-grapes-of-wrath-film/94-the-grapes-of-wrath-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991d_96-man-on-wire/96-man-on-wire.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_97-jaws-film/97-jaws-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_98-toy-story/98-toy-story.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_99-the-godfather-part-ii/99-the-godfather-part-ii.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_100-battleship-potemkin/100-battleship-potemkin.txt']

In [157]:
# Make directory if it doesn't already exist
folder_name = 'ebert_reviews'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [158]:
for url in ebert_review_urls:
    response = requests.get(url)
    with open(os.path.join(folder_name,url.split('/')[-1]),mode = 'wb') as file:
        file.write(response.content)

In [159]:
len(os.listdir(folder_name))

88

## Deal text file with encoding

[The Absolute Minimum Every Software Developer Absolutely, Positively Must Know About Unicode and Character Sets (No Excuses!)](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)

- ASCII 255 characters ,The traditional store-it-in-two-byte methods are called UCS-2
- then unicode U+0048 means H
- then utf-8 to save space,In UTF-8, every code point from 0-127 is stored in a single byte. Only code points 128 and above are stored using 2, 3, in fact, up to 6 bytes.that English text looks exactly the same in UTF-8 as it did in ASCII


UTF-8 or ASCII or ISO 8859-1 (Latin 1) or Windows 1252 (Western European), you simply cannot display it correctly or even figure out where it ends. There are over a hundred encodings and above code point 127,


Summary:

1. Unicode is the standard for computers to display and manipulate text while UTF-8 is one of the many mapping methods for Unicode
2. UTF-8 is a mapping method the retains compatibility with the older ASCII
3. UTF-8 is the most space efficient mapping method for Unicode compared to other encoding methods
4. UTF-8 is the most used Unicode standard for the web

Read more: Difference Between Unicode and UTF-8 | Difference Between http://www.differencebetween.net/technology/difference-between-unicode-and-utf-8/#ixzz68bedDUeJ

#### Unicode and Python
In Python 3, there is:

- one text type: str, which holds Unicode data and
- two byte types: bytes and bytearray

### Text Files in Python
- The first two minutes of the video below are dedicated to the glob library, which makes opening files with similar path structure (like our folder of Roger Ebert review text files) simple.

In [160]:
#This is one way to use os
import os
folder = 'ebert_reviews'
for ebert_review in os.listdir(folder):
    with open(os.path.join(folder,ebert_review)) as file:

SyntaxError: unexpected EOF while parsing (<ipython-input-160-ce942eebc6bb>, line 5)

In computer programming, glob patterns specify sets of filenames with wildcard characters. For example, the Unix Bash shell command mv *.txt textfiles/ moves (mv) all files with names ending in .txt from the current directory to the directory textfiles. Here, * is a wildcard standing for "any string of character

In [161]:
#there is another way to use glob,which makes opening files with similar path structure
import glob

In [178]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []

for ebert_reviews in glob.glob('ebert_reviews/*.txt'):
    with open(ebert_reviews, encoding = 'utf-8') as file:
        title = file.readline()[:-1]
        url = file.readline()[:-1]
        context = file.readline()
    df_list.append({'title':title,'review_url':url,'review_text':context})

df = pd.DataFrame(df_list)
   

In [179]:
df.head()

,title,review_url,review_text
0,Dunkirk (2017),http://www.rogerebert.com/reviews/dunkirk-2017,"Lean and ambitious, unsentimental and bombasti..."
1,Army of Shadows (L'Armée des ombres) (1969),http://www.rogerebert.com/reviews/great-movie-...,"Jean-Pierre Melville's ""Army of Shadows"" is ab..."
2,Alien (1979),http://www.rogerebert.com/reviews/great-movie-...,"At its most fundamental level, ""Alien"" is a mo..."
3,The Bride of Frankenstein (1935),http://www.rogerebert.com/reviews/great-movie-...,To a new world of gods and monsters.\n
4,The 400 Blows (Les Quatre cents coups) (1959),http://www.rogerebert.com/reviews/great-movie-...,I demand that a film express either the joy of...


## The correct, fully Pythonic way to read a file is the following:

`with open(...) as f:`

    `for line in f:`
    
       ` # Do something with 'line'`
        
The with statement handles opening and closing the file, including if an exception is raised in the inner block. The for line in f treats the file object f as an iterable, which automatically uses buffered I/O and memory management so you don't have to worry about large files.

There should be one -- and preferably only one -- obvious way to do it.

# Source: APIs (Application Programming Interfaces)
use APIs to get the image of each movie.

### MediaWiki API

MediaWiki has a great [tutorial](https://www.mediawiki.org/wiki/API:Tutorial) on their website on how their API calls are structured. It's a nice and simple example and they explain the various moving parts:

- The endpoint (important takeaway: there is nothing special about this URL!)
- The format
- The action
- Action-specific parameters

To search for page titles matching Craig Noone, the URL for the HTTP GET request would be:

api.php?action=query&list=search&srsearch=Craig%20Noone&format=jsonfm

Explanation of each part of the URL:

- http://en.wikipedia.org/w/api.php is the main endpoint. In this case it is English Wikipedia.
- action=query means fetch data from wiki.
- list=search means get list of pages matching a criteria.
- srsearch=Craig%20Noone indicates the page title to search for. The %20 indicates a space character in a URL.
- format=jsonfm indicates JSON output, which is the recommended output format.

There are a bunch of different access libraries for MediaWiki to satisfy the variety of programming languages that exist. Here is a list for Python. This is pretty standard for most APIs. Some libraries are better than others, which again, is standard. For a MediaWiki, the most up to date and human readable one in Python is called wptools. The analogous relationship for Twitter is:

- MediaWiki API → wptools
- Twitter API → tweepy

`Javascript Object Notation (JSON)`, emerged as a standard for easily exchanging Javscript object data between systems. 

`XML` – which is short for “eXtensible Markup Language”. 

In [2]:
import wptools

[install and import wptools](https://stackoverflow.com/questions/54134917/is-there-any-configuration-in-the-jupyter-notebook-that-prevents-the-import-wpto)

pip install -U wptools

In [3]:
page = wptools.page('E.T._the_Extra-Terrestrial').get()

en.wikipedia.org (query) E.T._the_Extra-Terrestrial
en.wikipedia.org (query) E.T. the Extra-Terrestrial (&plcontinue=...
en.wikipedia.org (parse) 73441
www.wikidata.org (wikidata) Q11621
www.wikidata.org (labels) P462|P3107|P3417|Q3414212|P1981|P244|P2...
www.wikidata.org (labels) Q60629803|P2047|Q6860813|Q130232|Q95291...
www.wikidata.org (labels) Q900414|P3844|P136|P2529|P1712|P2603|P4...
www.wikidata.org (labels) Q471839|Q2143665|Q8395520|P910|Q168383|...
en.wikipedia.org (restbase) /page/summary/E.T._the_Extra-Terrestrial
en.wikipedia.org (imageinfo) File:ET logo 3.svg|File:E t the extr...
E.T. the Extra-Terrestrial (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(2)> E.T., ET
  assessments: <dict(4)> United States, Film, Science Fiction, Lib...
  claims: <dict(98)> P1562, P57, P272, P345, P31, P161, P373, P480...
  description: <str(63)> 1982 American science fiction film direct...
  exhtml: <str(606)> <p><i><b>E.T. the Extra-Terrestrial</b></i> i...
  exrest: <str(585)

Access the first image in the images attribute, which is a JSON array.

In [42]:
page.data['image'][0]['url']

'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'

Access the director key of the infobox attribute, which is a JSON object.

In [25]:
page.data['infobox']

{'name': 'E.T. the Extra-Terrestrial',
 'image': 'E t the extra terrestrial ver3.jpg',
 'caption': 'Theatrical release poster by [[John Alvin]]',
 'director': '[[Steven Spielberg]]',
 'producers': '{{unbulleted list|[[Kathleen Kennedy (producer)|Kathleen Kennedy]]|Steven Spielberg}}',
 'writer': '[[Melissa Mathison]]',
 'starring': '{{Plainlist|<!--Per poster billing-->|\n* [[Dee Wallace]]\n* [[Peter Coyote]]\n* [[Henry Thomas]]}} * [[Dee Wallace]]\n* [[Peter Coyote]]\n* [[Henry Thomas]]',
 'music': '[[John Williams]]',
 'cinematography': '[[Allen Daviau]]',
 'editing': '[[Carol Littleton]]',
 'studio': '[[Universal Studios]]<br>[[Amblin Entertainment]]',
 'distributor': 'Universal Pictures',
 'released': '{{Film date|1982|05|26|[[1982 Cannes Film Festival|Cannes]]|1982|06|11|United States}}',
 'runtime': '114 minutes',
 'country': 'United States',
 'language': 'English',
 'budget': '$10.5 million',
 'gross': '$792.9 million'}

## Something about Json

In [9]:
import json

In [10]:
data = {}
data['people'] = []
data['people'].append({
    'name': 'Scott',
    'website': 'stackabuse.com',
    'from': 'Nebraska'
})
data['people'].append({
    'name': 'Larry',
    'website': 'google.com',
    'from': 'Michigan'
})
data['people'].append({
    'name': 'Tim',
    'website': 'apple.com',
    'from': 'Alabama'
})

In [11]:
with open('data.txt', 'w') as outfile:
    json.dump(data, outfile)

After importing the json library, we construct some simple data to write to our file. The important part comes at the end when we use the with statement to open our destination file, then use json.dump to write the data object to the outfile file.

A slight variation on the json.dump method that's worth mentioning is json.dumps, which returns the actual JSON string instead of sending it directly to a writable object. This can give you some more control if you need to make some changes to the JSON string (like encrypting it, for example).

### Reading JSON from a File
On the other end, reading JSON data from a file is just as easy as writing it to a file. Using the same json package again, we can extract and parse the JSON string directly from a file object. In the following example, we do just that and then print out the data we got:

In [12]:
with open('data.txt') as json_file:
    data = json.load(json_file)
    for p in data['people']:
        print('Name: ' + p['name'])
        print('Website: ' + p['website'])
        print('From: ' + p['from'])
        print('')

Name: Scott
Website: stackabuse.com
From: Nebraska

Name: Larry
Website: google.com
From: Michigan

Name: Tim
Website: apple.com
From: Alabama



json.load is the important method to note here. It reads the string from the file, parses the JSON data, populates a Python dict with the data and returns it back to you.

## Options
When serializing your data to JSON with Python, the result will be in the standard format and not very readable since whitespace is eliminated. While this is the ideal behavior for most cases, sometimes you may need to make small changes, like adding whitespace to make it human readable. Both json.dump and json.load provide quite a few options for more flexibility, a few of which will be described here.

#### Pretty-Printing
Making JSON human readable (aka "pretty printing") is as easy as passing an integer value for the indent parameter:

In [15]:
import json
data = {}
data['people'] = []
data['people'].append({
    'name': 'Scott',
    'website': 'stackabuse.com',
    'from': 'Nebraska'
})
data['people'].append({
    'name': 'Larry',
    'website': 'google.com',
    'from': 'Michigan'
})
data['people'].append({
    'name': 'Tim',
    'website': 'apple.com',
    'from': 'Alabama'
})
print(json.dumps(data, indent=4))

{
    "people": [
        {
            "name": "Scott",
            "website": "stackabuse.com",
            "from": "Nebraska"
        },
        {
            "name": "Larry",
            "website": "google.com",
            "from": "Michigan"
        },
        {
            "name": "Tim",
            "website": "apple.com",
            "from": "Alabama"
        }
    ]
}


This is actually quite useful since you'll often have to read JSON data during development. Another option is to use the command line tool, json.tool. So if you just want to pretty-print JSON to the command line you can do something like this:

## Sorting
In JSON, an object is defined as:

#### An object is an unordered set of name/value pairs.

So the standard is saying that key order isn't guaranteed, but it's possible that you may need it for your own purposes internally. To achieve ordering, you can pass True to the sort_keys option when using `json.dump` or `json.dumps`.

In [17]:
print(json.dumps(data, sort_keys=True, indent=4))

{
    "people": [
        {
            "from": "Nebraska",
            "name": "Scott",
            "website": "stackabuse.com"
        },
        {
            "from": "Michigan",
            "name": "Larry",
            "website": "google.com"
        },
        {
            "from": "Alabama",
            "name": "Tim",
            "website": "apple.com"
        }
    ]
}


### ASCII Text
By default, json.dump will ensure that all of your text in the given Python dictionary are ASCII-encoded. If non-ASCII characters are present, then they're automatically escaped,

In [20]:
data2 = {'item': 'Beer', 'cost':'£4.00'}
jstr = json.dumps(data2, indent=4)
print(jstr)

{
    "item": "Beer",
    "cost": "\u00a34.00"
}


This isn't always acceptable, and in many cases you may want to keep your Unicode characters un-touched. To do this, set the ensure_ascii option to False.

In [21]:
jstr = json.dumps(data2, ensure_ascii=False, indent=4)
print(jstr)

{
    "item": "Beer",
    "cost": "£4.00"
}


### json.dump(load) is about files, js.dumps(loads) is used to change stracture

## Mashup: APIs, Downloading Files Programmatically, and JSON
With APIs, downloading files programmatically from the internet, and JSON under your belt, you now have all of the knowledge to download all of the movie poster images for the Roger Ebert review word clouds. This is your next task.

There are two key things to be aware of before you begin:

### 1. Wikipedia Page Titles
To access Wikipedia page data via the MediaWiki API with wptools (phew, that was a mouthful), you need each movie's Wikipedia page title, i.e., what comes after the last slash in en.wikipedia.org/wiki/ in the URL. For this lesson, I've compiled all of these titles for each of the movies in the Top 100 for you.

### 2. Downloading Image Files
Downloading images may seem tricky from a reading and writing perspective, in comparison to text files which you can read line by line, for example. But in reality, image files aren't special—they're just binary files. To interact with them, you don't need special software (like Photoshop or something) that "understands" images. You can use regular file opening, reading, and writing techniques, like this:

`import requests
r = requests.get(url)
with open(folder_name + '/' + filename, 'wb') as f:
        f.write(r.content)`
        
But this technique can be error-prone. It will work most of the time, but sometimes the file you write to will be damaged. 

This type of error is why the requests library maintainers recommend using the PIL library (short for Pillow) and BytesIO from the io library for non-text requests, like images. They recommend that you access the response body as bytes, for non-text requests. For example, to create an image from binary data returned by a request:

`import requests
from PIL import Image
from io import BytesIO
r = requests.get(url)
i = Image.open(BytesIO(r.content))`

Though you may still encounter a similar file error, this code above will at least warn us with an error message, at which point we can manually download the problematic images.

### Quiz
Let's gather the last piece of data for the Roger Ebert review word clouds now: the movie poster image files. Let's also keep each image's URL to add to the master DataFrame later.

Though we're going to use a loop to minimize repetition, here's how the major parts inside that loop will work, in order:

1. We're going to query the MediaWiki API using wptools to get a movie poster URL via each page object's image attribute.
2. Using that URL, we'll programmatically download that image into a folder called bestofrt_posters.

0) import pandas,wptools,os,requests,Image(PIL),BytesIO(io)

In [23]:
import pandas as pd
import wptools
import os
import requests
from PIL import Image
from io import BytesIO

1) Contains title_list, which is a list of all of the Wikipedia page titles for each movie in the Rotten Tomatoes Top 100 Movies of All Time list. This list is in the same order as the Top 100.

In [24]:
title_list = [
 'The_Wizard_of_Oz_(1939_film)',
 'Citizen_Kane',
 'The_Third_Man',
 'Get_Out_(film)',
 'Mad_Max:_Fury_Road',
 'The_Cabinet_of_Dr._Caligari',
 'All_About_Eve',
 'Inside_Out_(2015_film)',
 'The_Godfather',
 'Metropolis_(1927_film)',
 'E.T._the_Extra-Terrestrial',
 'Modern_Times_(film)',
 'It_Happened_One_Night',
 "Singin'_in_the_Rain",
 'Boyhood_(film)',
 'Casablanca_(film)',
 'Moonlight_(2016_film)',
 'Psycho_(1960_film)',
 'Laura_(1944_film)',
 'Nosferatu',
 'Snow_White_and_the_Seven_Dwarfs_(1937_film)',
 "A_Hard_Day%27s_Night_(film)",
 'La_Grande_Illusion',
 'North_by_Northwest',
 'The_Battle_of_Algiers',
 'Dunkirk_(2017_film)',
 'The_Maltese_Falcon_(1941_film)',
 'Repulsion_(film)',
 '12_Years_a_Slave_(film)',
 'Gravity_(2013_film)',
 'Sunset_Boulevard_(film)',
 'King_Kong_(1933_film)',
 'Spotlight_(film)',
 'The_Adventures_of_Robin_Hood',
 'Rashomon',
 'Rear_Window',
 'Selma_(film)',
 'Taxi_Driver',
 'Toy_Story_3',
 'Argo_(2012_film)',
 'Toy_Story_2',
 'The_Big_Sick',
 'Bride_of_Frankenstein',
 'Zootopia',
 'M_(1931_film)',
 'Wonder_Woman_(2017_film)',
 'The_Philadelphia_Story_(film)',
 'Alien_(film)',
 'Bicycle_Thieves',
 'Seven_Samurai',
 'The_Treasure_of_the_Sierra_Madre_(film)',
 'Up_(2009_film)',
 '12_Angry_Men_(1957_film)',
 'The_400_Blows',
 'Logan_(film)',
 'All_Quiet_on_the_Western_Front_(1930_film)',
 'Army_of_Shadows',
 'Arrival_(film)',
 'Baby_Driver',
 'A_Streetcar_Named_Desire_(1951_film)',
 'The_Night_of_the_Hunter_(film)',
 'Star_Wars:_The_Force_Awakens',
 'Manchester_by_the_Sea_(film)',
 'Dr._Strangelove',
 'Frankenstein_(1931_film)',
 'Vertigo_(film)',
 'The_Dark_Knight_(film)',
 'Touch_of_Evil',
 'The_Babadook',
 'The_Conformist_(film)',
 'Rebecca_(1940_film)',
 "Rosemary%27s_Baby_(film)",
 'Finding_Nemo',
 'Brooklyn_(film)',
 'The_Wrestler_(2008_film)',
 'The_39_Steps_(1935_film)',
 'L.A._Confidential_(film)',
 'Gone_with_the_Wind_(film)',
 'The_Good,_the_Bad_and_the_Ugly',
 'Skyfall',
 'Rome,_Open_City',
 'Tokyo_Story',
 'Hell_or_High_Water_(film)',
 'Pinocchio_(1940_film)',
 'The_Jungle_Book_(2016_film)',
 'La_La_Land_(film)',
 'Star_Trek_(film)',
 'High_Noon',
 'Apocalypse_Now',
 'On_the_Waterfront',
 'The_Wages_of_Fear',
 'The_Last_Picture_Show',
 'Harry_Potter_and_the_Deathly_Hallows_–_Part_2',
 'The_Grapes_of_Wrath_(film)',
 'Roman_Holiday',
 'Man_on_Wire',
 'Jaws_(film)',
 'Toy_Story',
 'The_Godfather_Part_II',
 'Battleship_Potemkin'
]

2) Creates an empty list, df_list, to which dictionaries will be appended. This list of dictionaries will eventually be converted to a pandas DataFrame

In [51]:
df_list = []

3) Creates an empty folder, bestofrt_posters, to store the downloaded movie poster image files.

In [52]:
folder_name = 'bestofrt_posters'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

4) Creates an empty dictionary, image_errors, to fill to keep track of movie poster image URLs that don't work.

In [53]:
image_errors = {}

5) Loops through the Wikipedia page titles in title_list and:
- Stores the ranking of that movie in the Top 100 list based on its position in title_list. Ranking is needed so we can join this DataFrame with the master DataFrame later. We can't join on title because the titles of the Rotten Tomatoes pages and the Wikipedia pages differ.
- Uses try and except blocks to attempt to query MediaWiki for a movie poster image URL and to attempt to download that image. If the attempt fails and an error is encountered, the offending movie is documented in image_errors.
- Appends a dictionary with ranking, title, and poster_url as the keys and the extracted values for each as the values to df_list.

In [54]:
x = 0
while x < len(title_list):
    try:
        title = title_list[x]
        ranking = x+1
        # This cell is slow so print ranking to gauge time remaining
        print(ranking)
        page = wptools.page(title, silent=True)
        images = page.get()
        first_image_url = images.data['image'][0]['url']
#        print(first_image_url)
        r = requests.get(first_image_url)
#        print(r)
        # Download movie poster image
        i = Image.open(BytesIO(r.content))
        image_file_format = first_image_url.split('.')[-1]
        i.save(folder_name + "/" + str(ranking) + "_" + title + '.' + image_file_format)
        
        # Append to list of dictionaries
        df_list.append({'ranking': ranking,
                        'title': title,
                        'poster_url': first_image_url})

    # Not best practice to catch all exceptions but fine for this short script
    except Exception as e:
        print(str(ranking) + "_" + title + ": " + str(e))
        image_errors[str(ranking) + "_" + title] = images
    
    
    
    
    
    
    x += 1
    print(df_list)



    

1
[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}]
2
[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}]
3
[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}]
4
[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZA

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

API error: {'code': 'invalidtitle', 'info': 'Bad title "A_Hard_Day%27s_Night_(film)".', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


...

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

[{'ranking': 1, 'title': 'The_Wizard_of_Oz_(1939_film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/commons/c/ca/WIZARD_OF_OZ_ORIGINAL_POSTER_1939.jpg'}, {'ranking': 2, 'title': 'Citizen_Kane', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/c/ce/Citizenkane.jpg'}, {'ranking': 3, 'title': 'The_Third_Man', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/21/ThirdManUSPoster.jpg'}, {'ranking': 4, 'title': 'Get_Out_(film)', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/a/a3/Get_Out_poster.png'}, {'ranking': 5, 'title': 'Mad_Max:_Fury_Road', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/6/6e/Mad_Max_Fury_Road.jpg'}, {'ranking': 6, 'title': 'The_Cabinet_of_Dr._Caligari', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/2f/The_Cabinet_of_Dr._Caligari_poster.jpg'}, {'ranking': 7, 'title': 'All_About_Eve', 'poster_url': 'https://upload.wikimedia.org/wikipedia/en/2/22/AllAboutEve.jpeg'}, {'ranking': 8, 'title': 'Inside_Out_(2015_film

In [55]:
image_errors

{'22_A_Hard_Day%27s_Night_(film)': <wptools.page.WPToolsPage at 0x104546c10>,
 '64_Dr._Strangelove': <wptools.page.WPToolsPage at 0x115928390>,
 '72_Rosemary%27s_Baby_(film)': <wptools.page.WPToolsPage at 0x115d8c690>,
 '83_Hell_or_High_Water_(film)': <wptools.page.WPToolsPage at 0x104719910>}

6) Inspects the images that caused errors and downloads the correct image individually (either via another URL in the image attribute's list or a URL from Google Images)

In [56]:
for key in image_errors.keys():
    print(key)

22_A_Hard_Day%27s_Night_(film)
64_Dr._Strangelove
72_Rosemary%27s_Baby_(film)
83_Hell_or_High_Water_(film)


In [60]:
# Inspect unidentifiable images and download them individually
for rank_title, images in image_errors.items():
    if rank_title == '22_A_Hard_Day%27s_Night_(film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/4/47/A_Hard_Days_night_movieposter.jpg'
#    if rank_title == '64_Dr._Strangelove':
#        url = 'https://en.wikipedia.org/wiki/Dr._Strangelove#/media/File:Dr._Strangelove_poster.jpg'
    if rank_title == '72_Rosemary%27s_Baby_(film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/e/ef/Rosemarys_baby_poster.jpg'
    if rank_title == '83_Hell_or_High_Water_(film)':
        url = 'https://en.wikipedia.org/wiki/Hell_or_High_Water_(2016_film)#/media/File:Hell_or_High_Water_film_poster.png'
    title = rank_title[3:]
    print(title)
    df_list.append({'ranking': int(title_list.index(title) + 1),
                    'title': title,
                    'poster_url': url})
    r = requests.get(url)
    # Download movie poster image
    i = Image.open(BytesIO(r.content))
    image_file_format = url.split('.')[-1]
    i.save(folder_name + "/" + rank_title + '.' + image_file_format)

A_Hard_Day%27s_Night_(film)
Dr._Strangelove
Rosemary%27s_Baby_(film)
Hell_or_High_Water_(film)


OSError: cannot identify image file <_io.BytesIO object at 0x1045aef50>

7) Creates a DataFrame called df by converting df_list using the pd.DataFrame

In [62]:
# Create DataFrame from list of dictionaries
df = pd.DataFrame(df_list, columns = ['ranking', 'title', 'poster_url'])
df = df.sort_values('ranking').reset_index(drop=True)
df

,ranking,title,poster_url
0,1,The_Wizard_of_Oz_(1939_film),https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen_Kane,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The_Third_Man,https://upload.wikimedia.org/wikipedia/en/2/21...
3,4,Get_Out_(film),https://upload.wikimedia.org/wikipedia/en/a/a3...
4,5,Mad_Max:_Fury_Road,https://upload.wikimedia.org/wikipedia/en/6/6e...
...,...,...,...
99,96,Man_on_Wire,https://upload.wikimedia.org/wikipedia/en/5/54...
100,97,Jaws_(film),https://upload.wikimedia.org/wikipedia/en/e/eb...
101,98,Toy_Story,https://upload.wikimedia.org/wikipedia/en/1/13...
102,99,The_Godfather_Part_II,https://upload.wikimedia.org/wikipedia/en/0/03...


In [64]:
df.duplicated().sum()

3

In [67]:
df.drop_duplicates(inplace = True)

In [69]:
df.duplicated().sum()

0

In [70]:
df.shape

(101, 3)

In [71]:
df.reset_index(drop=True)

,ranking,title,poster_url
0,1,The_Wizard_of_Oz_(1939_film),https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen_Kane,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The_Third_Man,https://upload.wikimedia.org/wikipedia/en/2/21...
3,4,Get_Out_(film),https://upload.wikimedia.org/wikipedia/en/a/a3...
4,5,Mad_Max:_Fury_Road,https://upload.wikimedia.org/wikipedia/en/6/6e...
...,...,...,...
96,96,Man_on_Wire,https://upload.wikimedia.org/wikipedia/en/5/54...
97,97,Jaws_(film),https://upload.wikimedia.org/wikipedia/en/e/eb...
98,98,Toy_Story,https://upload.wikimedia.org/wikipedia/en/1/13...
99,99,The_Godfather_Part_II,https://upload.wikimedia.org/wikipedia/en/0/03...


In [80]:
df.title.duplicated().sum()

0

In [74]:
df[df.title.duplicated()]

,ranking,title,poster_url
66,64,Dr._Strangelove,https://en.wikipedia.org/wiki/Dr._Strangelove#...


In [75]:
df.drop(66,inplace = True)

In [76]:
df

,ranking,title,poster_url
0,1,The_Wizard_of_Oz_(1939_film),https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen_Kane,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The_Third_Man,https://upload.wikimedia.org/wikipedia/en/2/21...
3,4,Get_Out_(film),https://upload.wikimedia.org/wikipedia/en/a/a3...
4,5,Mad_Max:_Fury_Road,https://upload.wikimedia.org/wikipedia/en/6/6e...
...,...,...,...
99,96,Man_on_Wire,https://upload.wikimedia.org/wikipedia/en/5/54...
100,97,Jaws_(film),https://upload.wikimedia.org/wikipedia/en/e/eb...
101,98,Toy_Story,https://upload.wikimedia.org/wikipedia/en/1/13...
102,99,The_Godfather_Part_II,https://upload.wikimedia.org/wikipedia/en/0/03...


In [77]:
df.reset_index(drop=True,inplace = True)

In [78]:
df

,ranking,title,poster_url
0,1,The_Wizard_of_Oz_(1939_film),https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen_Kane,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The_Third_Man,https://upload.wikimedia.org/wikipedia/en/2/21...
3,4,Get_Out_(film),https://upload.wikimedia.org/wikipedia/en/a/a3...
4,5,Mad_Max:_Fury_Road,https://upload.wikimedia.org/wikipedia/en/6/6e...
...,...,...,...
95,96,Man_on_Wire,https://upload.wikimedia.org/wikipedia/en/5/54...
96,97,Jaws_(film),https://upload.wikimedia.org/wikipedia/en/e/eb...
97,98,Toy_Story,https://upload.wikimedia.org/wikipedia/en/1/13...
98,99,The_Godfather_Part_II,https://upload.wikimedia.org/wikipedia/en/0/03...


In [79]:
df.to_csv('df.csv',index = False)

# Storing Data
Storing is usually done after cleaning, but it's not always done, which excludes it from being a core part of the data wrangling process. Sometimes you just analyze and visualize and leave it at that, without saving your new data.

The advantages and disadvantages of flat files were discussed earlier in the lesson in the Flat File Structure concept. One of the advantages:

- Great for small datasets.

And one of the disadvantages:

- Sharing data can be cumbersome.

## Relational Databases in Python
### Data Wrangling and Relational Databases
In the context of data wrangling, we recommend that databases and SQL only come into play for gathering data or storing data. That is:

- **Connecting to a database and importing data** into a pandas DataFrame (or the analogous data structure in your preferred programming language), then assessing and cleaning that data, or
- **Connecting to a database and storing data** you just gathered (which could potentially be from a database), assessed, and cleaned

These tasks are especially necessary when you have large amounts of data, which is where SQL and other databases excel over flat files.

The two scenarios above can be further broken down into three main tasks:

- Connecting to a database in Python
- Storing data **from** a pandas DataFrame in a database to which you're connected, and
- Importing data **from** a database to which you're connected to a pandas DataFrame

1. Connect to a database. We'll connect to a SQLite database using [SQLAlchemy](https://www.sqlalchemy.org/), a database toolkit for Python.
2. Store the data in the cleaned master dataset in that database. We'll do this using pandas' `to_sql` DataFrame method.
3. Then read the brand new data in that database back into a pandas DataFrame. We'll do this using pandas' `read_sql` function.

In [82]:
import pandas as pd

In [90]:
pd.options.display.max_rows

60

Imagine this notebook contains all of the gathering code from this entire lesson, plus the assessing and cleaning code done behind the scenes, and that the final product is a merged master DataFrame called df.

In [83]:
df = pd.read_csv('bestofrt_master.csv')

In [91]:
df.head(60)

,ranking,title,critic_score,number_of_critic_ratings,audience_score,number_of_audience_ratings,review_url,review_text,poster_url
0,1,The Wizard of Oz (1939),99,110,89,874425,http://www.rogerebert.com/reviews/great-movie-...,As a child I simply did not notice whether a m...,https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen Kane (1941),100,75,90,157274,http://www.rogerebert.com/reviews/great-movie-...,“I don't think any word can explain a man's li...,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The Third Man (1949),100,77,93,53081,http://www.rogerebert.com/reviews/great-movie-...,Has there ever been a film where the music mor...,https://upload.wikimedia.org/wikipedia/en/2/21...
3,4,Get Out (2017),99,282,87,63837,http://www.rogerebert.com/reviews/get-out-2017,"With the ambitious and challenging “Get Out,” ...",https://upload.wikimedia.org/wikipedia/en/e/eb...
4,5,Mad Max: Fury Road (2015),97,370,86,123937,http://www.rogerebert.com/reviews/mad-max-fury...,George Miller’s “Mad Max” films didn’t just ma...,https://upload.wikimedia.org/wikipedia/en/6/6e...
5,6,The Cabinet of Dr. Caligari (Das Cabinet des D...,100,49,89,27163,http://www.rogerebert.com/reviews/great-movie-...,The first thing everyone notices and best reme...,https://upload.wikimedia.org/wikipedia/commons...
6,7,All About Eve (1950),100,64,94,44564,http://www.rogerebert.com/reviews/great-movie-...,Growing older was a smart career move for Bett...,https://upload.wikimedia.org/wikipedia/en/2/22...
7,8,Inside Out (2015),98,324,89,133558,http://www.rogerebert.com/reviews/inside-out-2015,"""Inside Out,"" a comedy-adventure set inside th...",https://upload.wikimedia.org/wikipedia/en/0/0a...
8,9,The Godfather (1972),99,85,98,730433,http://www.rogerebert.com/reviews/great-movie-...,“The Godfather” is told entirely within a clos...,https://upload.wikimedia.org/wikipedia/en/1/1c...
9,10,Metropolis (1927),99,116,92,62018,http://www.rogerebert.com/reviews/great-movie-...,The opening shots of the restored “Metropolis”...,https://upload.wikimedia.org/wikipedia/en/0/06...


### 1. Connect to a database

In [85]:
from sqlalchemy import create_engine

In [86]:
# Create SQLAlchemy Engine and empty bestofrt database
# bestofrt.db will not show up in the Jupyter Notebook dashboard yet
engine = create_engine('sqlite:///bestofrt.db')

### 2. Store pandas DataFrame in database
Store the data in the cleaned master dataset (bestofrt_master) in that database.

In [87]:
# Store cleaned master DataFrame ('df') in a table called master in bestofrt.db
# bestofrt.db will be visible now in the Jupyter Notebook dashboard
df.to_sql('master', engine, index=False)

### 3. Read database data into a pandas DataFrame
Read the brand new data in that database back into a pandas DataFrame.

In [88]:
df_gather = pd.read_sql('SELECT * FROM master', engine)

In [89]:
df_gather.head(3)

,ranking,title,critic_score,number_of_critic_ratings,audience_score,number_of_audience_ratings,review_url,review_text,poster_url
0,1,The Wizard of Oz (1939),99,110,89,874425,http://www.rogerebert.com/reviews/great-movie-...,As a child I simply did not notice whether a m...,https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen Kane (1941),100,75,90,157274,http://www.rogerebert.com/reviews/great-movie-...,“I don't think any word can explain a man's li...,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The Third Man (1949),100,77,93,53081,http://www.rogerebert.com/reviews/great-movie-...,Has there ever been a film where the music mor...,https://upload.wikimedia.org/wikipedia/en/2/21...


## Other File Formats
The types of files you mastered in this lesson are the ones you'll interact with for the vast majority of your wrangling projects in the future. Again, these were:

- Flat files (e.g. CSV and TSV)
- HTML files
- JSON files
- TXT files
- Relational database files

Additional, less common file formats include:

- [Excel files](https://www.lifewire.com/what-is-an-xlsx-file-2622540)
- [Pickle files](https://stackoverflow.com/questions/7501947/understanding-pickling-in-python)
- [HDF5 files](https://www.neonscience.org/resources/data-tutorials)
- [SAS files](https://whatis.techtarget.com/fileformat/SAS-SAS-program-file)
- [STATA files](http://faculty.econ.ucdavis.edu/faculty/cameron/stata/stataintro.html)

pandas has [functions](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) to read (and write, to most of them) these files. 